In [1]:
import pandas as pd
import numpy as np
from elements import elements
import elements.elements as elements
from astropy.table import Table

In [2]:
vdat = pd.read_csv('VALD_linelist_raw', header=None, skiprows=2, nrows=400000)
vdat = vdat.drop(13,axis=1)
vdatc = vdat.copy()
vdat = vdat[vdat[12].notna()]
vdat = vdat[0:100000]
vdat = vdat.reset_index(drop=True)
#vdat.iloc[92717]
vdat

/home/katrine/anaconda3/envs/carsus/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,'Sc 5',50.000650,-3.096,263453.10,0.5,463450.500,0.5,0.67,2.66,1.66,9.71,-7.33,-8.09
1,'Ni 3',50.001630,-4.145,2269.60,2,202263.080,1.0,0.67,0.14,0.93,8.36,-6.70,-7.72
2,'Co 4',50.002225,-2.220,29592.20,4,229583.300,5.0,1.05,1.00,0.90,9.32,-7.08,-8.02
3,'Mg 4',50.002400,-4.340,597065.70,1.5,797056.000,0.5,99.00,99.00,99.00,0.00,0.00,0.00
4,'Ca 2',50.003533,-4.775,89214.13,0.5,289200.000,0.5,1.56,2.00,1.78,9.13,-3.73,-6.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,'Ni 5',125.034384,-0.250,216596.00,3.0,296574.000,3.0,1.51,1.47,1.49,9.74,-6.94,-8.06
99996,'Fe 2',125.034827,-3.578,18360.64,1.5,98338.357,1.5,1.28,1.77,1.52,8.70,-6.52,-7.81
99997,'CO 1',125.034979,-3.206,1059.37,23.0,81036.990,24.0,99.00,99.00,99.00,8.05,0.00,0.00
99998,'Cr 4',125.035166,-0.146,161495.30,2.5,241472.800,3.5,1.26,1.24,1.22,9.76,-6.51,-7.89


In [3]:
el_list = pd.read_csv('elements.txt',delimiter=' ',header=None)
numbers = range(len(el_list[0])+1)
el_dict = dict(zip(el_list[0], numbers[1:]))

el = vdat[0].str.split(" ", n = 1, expand = True)
el[0] = el[0].replace({'\'':''}, regex=True)

for i in range(len(el[0])):
    e = el[0][i]
    if e in el_dict:
        el[0][i] = el_dict[e]
    else:
        el[0][i] = 0

el[1] = (el[1].replace({'\'':''}, regex=True).astype(int) - 1).apply(lambda x: '{0:0>2}'.format(x))
elm = el[0].map(str) + '.' + el[1].map(str)

In [4]:
labels = vdatc.copy()
labels = labels[~labels[12].notna()][0:300000]
labels = pd.Series(labels.fillna('').values.tolist()).str.join(',')
labels = labels.replace({',,+':''}, regex=True)
labels = labels.replace({'\'  ':''}, regex=True)
labels = labels.replace({'\'':''}, regex=True)
labels = labels.replace({'  +':' '}, regex=True)

label_l = labels.iloc[::3]
label_l = label_l.reset_index(drop=True)
label_u = labels.iloc[1::3]
label_u = label_u.reset_index(drop=True)

In [5]:
wave = vdat[1].astype(float).round(4)
log_gf = (vdat[2].astype(float)).round(3)
E_l = vdat[3].astype(float)
j_l = vdat[4].apply(lambda x: '{0:0>2}'.format(x))
#label_l = pd.DataFrame(['2D' for s in range(len(vdat))])
E_u = vdat[5].astype(float)
j_u = vdat[6].apply(lambda x: '{0:0>2}'.format(x))
#label_u = pd.DataFrame(['2D' for s in range(len(vdat))])
lande_l = vdat[7]*1000
lande_u = vdat[8]*1000
rad = vdat[10]
stark = vdat[11]
waals = vdat[12]
zeroes = pd.DataFrame(np.zeros((len(vdat), 1)))

df_vald = pd.concat([wave, log_gf, elm, E_l, j_l, label_l, E_u, j_u, label_u, rad, stark, waals, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, zeroes, lande_l, lande_u], axis=1)
df_vald.columns = range(df_vald.shape[1])

In [6]:
df_vald = df_vald[~(df_vald[2].astype(float)<1)]
df_vald.to_csv('VALD_linelist.dat', header=None, index=None)

In [7]:
# pd.options.display.max_columns = df_vald.shape[1]
# pd.set_option('display.expand_frame_repr', False)

with open('VALD_linelist.dat', 'w') as f: 
    df_vald.to_string(f, col_space=10, index=False, header=None)